<a href="https://colab.research.google.com/github/Mackenzzy/machinelearning/blob/main/estudoAprendizagemDeMaquina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#instalação da lib e abrindo seção no spark
!pip install pyspark==3.3.1
from pyspark.sql import SparkSession

sc = SparkSession \
    .builder \
    .master('local[*]') \
    .getOrCreate()

In [ ]:
#download do db sobre o covid, fiz separado pois é um db gigante

!gdown 1-lSo7A-SIuntho9tAqVO25enmLlxFe7S
!gdown 1-iG4e4-SzJONLi3Nw98nm9UdFzoqCHDI

Downloading...
From: https://drive.google.com/uc?id=1-lSo7A-SIuntho9tAqVO25enmLlxFe7S
To: /content/treinamento.csv
100% 26.1M/26.1M [00:00<00:00, 47.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-iG4e4-SzJONLi3Nw98nm9UdFzoqCHDI
To: /content/teste.csv
100% 37.4M/37.4M [00:00<00:00, 56.9MB/s]


In [ ]:
# criando os dados para servir de treino
dfTreino = sc.read\
    .option('delimiter',',')\
    .option('header','true')\
    .option('inferschema','true')\
    .csv('file:////content/treinamento.csv')

# criando os dados para ser usados no teste
dfTeste = sc.read\
    .option('delimiter',',')\
    .option('header','true')\
    .option('inferschema','true')\
    .csv('file:////content/teste.csv')

In [ ]:
#nome da coluna \ tipo do dado

dfTreino.printSchema()

root
 |-- hora: double (nullable = true)
 |-- minuto: double (nullable = true)
 |-- temp_minima: double (nullable = true)
 |-- temp_maxima: double (nullable = true)
 |-- latitude_media: double (nullable = true)
 |-- longitude_media: double (nullable = true)
 |-- Classe: string (nullable = true)



In [ ]:
#lib para os id's
from pyspark.sql.functions import monotonically_increasing_id
#como a tabela não apresentou a coluna id(padrão no powerbi) eu adicionei
#para vir no indice 1 ao invés do 0 precisa ser colocado o '+1' depois dos ()
dfTeste = dfTeste.withColumn('id', monotonically_increasing_id() + 1)

#mostra as N primeiras linhas
dfTeste.show(8)

+---------+---------+-----------+-----------+--------------+---------------+--------+---+
|     hora|   minuto|temp_minima|temp_maxima|latitude_media|longitude_media|  Classe| id|
+---------+---------+-----------+-----------+--------------+---------------+--------+---+
|11.787212|29.005116|      1.123|     28.606|      34.56035|      136.21983|Moderado|  1|
|11.482636| 29.84576|      1.038|  29.306004|      34.56078|      136.22015|Moderado|  2|
|11.312064|29.751745|      1.123|  29.006002|      34.55864|      136.22047|Moderado|  3|
|11.530982|29.505793|      1.404|  29.081003|     34.560654|      136.22006|Moderado|  4|
|  11.4008|29.422289|      1.504|  29.181004|     34.559814|      136.22006|Moderado|  5|
| 11.50784| 29.18513|      1.423|  29.106003|     34.559834|      136.21964|Moderado|  6|
|11.318713|29.743177|      1.323|  28.806002|     34.560326|       136.2207|Moderado|  7|
|11.324524| 30.05444|      1.155|  29.006002|     34.560883|      136.21967|Moderado|  8|
+---------

In [ ]:
#mesma coisa acima porém para o df de treino
dfTreino = dfTreino.withColumn('id', monotonically_increasing_id() + 1)

#mostra as N primeiras linhas
dfTreino.show(8)

+---------+---------+-----------+-----------+--------------+---------------+--------+---+
|     hora|   minuto|temp_minima|temp_maxima|latitude_media|longitude_media|  Classe| id|
+---------+---------+-----------+-----------+--------------+---------------+--------+---+
|11.312758|30.169239|     -1.859|     27.495|     36.169994|      139.23022|Moderado|  1|
|11.292323|29.638779|   8.542997|  36.177994|     31.349113|       73.50964|  Quente|  2|
|11.600304| 29.64275|     -1.861|  27.695002|      36.17158|      139.22937|Moderado|  3|
|11.462425|30.157314|   9.776998|  36.077995|     31.351593|        73.5104|  Quente|  4|
|11.735474|  29.9684|     -1.662|  27.695002|     36.169456|      139.23021|Moderado|  5|
|11.544413| 30.08596|  15.882999|  33.014996|     23.310606|      88.250885|  Alerta|  6|
|11.614842|29.549337|  11.020995|   36.58599|      26.52031|       78.81148|  Alerta|  7|
|11.699689|29.373444|  23.332996|  28.202002|      0.800029|      110.08771|  Alerta|  8|
+---------

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler  # Importa as classes StringIndexer e VectorAssembler do módulo de recursos do PySpark.

#colunas que a maquina vai estudar
features = ['hora', 'minuto', 'temp_minima', 'temp_maxima', 'latitude_media', 'longitude_media']

#usei o stringindexer pra transformar dados de string em numeros pra ficar mais facil para os algoritmos de aprendizagem de maquina
dfTreino = StringIndexer(inputCol='Classe', outputCol='label').fit(dfTreino).transform(dfTreino)

# nesse caso especifico não mesclei colunas, mas por convenção usei o vectorAssembler pra ficar mais facil a leitura
dfTreino = VectorAssembler(inputCols=features, outputCol='features')\
    .transform(dfTreino)

#exibe as 50 linhas iniciais
dfTreino.show(50)

#nao executar essa célula novamente, pois toda vez que rodar vai tentar criar as colunas novamente

+---------+---------+-----------+-----------+--------------+---------------+--------+---+-----+--------------------+
|     hora|   minuto|temp_minima|temp_maxima|latitude_media|longitude_media|  Classe| id|label|            features|
+---------+---------+-----------+-----------+--------------+---------------+--------+---+-----+--------------------+
|11.312758|30.169239|     -1.859|     27.495|     36.169994|      139.23022|Moderado|  1|  0.0|[11.312758,30.169...|
|11.292323|29.638779|   8.542997|  36.177994|     31.349113|       73.50964|  Quente|  2|  3.0|[11.292323,29.638...|
|11.600304| 29.64275|     -1.861|  27.695002|      36.17158|      139.22937|Moderado|  3|  0.0|[11.600304,29.642...|
|11.462425|30.157314|   9.776998|  36.077995|     31.351593|        73.5104|  Quente|  4|  3.0|[11.462425,30.157...|
|11.735474|  29.9684|     -1.662|  27.695002|     36.169456|      139.23021|Moderado|  5|  0.0|[11.735474,29.968...|
|11.544413| 30.08596|  15.882999|  33.014996|     23.310606|    

In [ ]:
dfTreino.show(5)

+---------+---------+-----------+-----------+--------------+---------------+--------+---+-----+--------------------+
|     hora|   minuto|temp_minima|temp_maxima|latitude_media|longitude_media|  Classe| id|label|            features|
+---------+---------+-----------+-----------+--------------+---------------+--------+---+-----+--------------------+
|11.312758|30.169239|     -1.859|     27.495|     36.169994|      139.23022|Moderado|  1|  0.0|[11.312758,30.169...|
|11.292323|29.638779|   8.542997|  36.177994|     31.349113|       73.50964|  Quente|  2|  3.0|[11.292323,29.638...|
|11.600304| 29.64275|     -1.861|  27.695002|      36.17158|      139.22937|Moderado|  3|  0.0|[11.600304,29.642...|
|11.462425|30.157314|   9.776998|  36.077995|     31.351593|        73.5104|  Quente|  4|  3.0|[11.462425,30.157...|
|11.735474|  29.9684|     -1.662|  27.695002|     36.169456|      139.23021|Moderado|  5|  0.0|[11.735474,29.968...|
+---------+---------+-----------+-----------+--------------+----

In [ ]:
#se executar por engano as celuas acima :D
#dfTreino = dfTreino.drop('label')
#dfTreino = dfTreino.drop('features2')


In [ ]:
# agora modelei o df de teste

from pyspark.ml.feature import StringIndexer, VectorAssembler

features = ['hora', 'minuto', 'temp_minima', 'temp_maxima', 'latitude_media', 'longitude_media']

dfTeste = StringIndexer(inputCol='Classe', outputCol='label')\
    .fit(dfTeste)\
    .transform(dfTeste)

dfTeste = VectorAssembler(inputCols=features, outputCol='features')\
    .transform(dfTeste)

In [ ]:
dfTeste.show(5)

dfTreino.show(5)


+---------+---------+-----------+-----------+--------------+---------------+--------+---+-----+--------------------+
|     hora|   minuto|temp_minima|temp_maxima|latitude_media|longitude_media|  Classe| id|label|            features|
+---------+---------+-----------+-----------+--------------+---------------+--------+---+-----+--------------------+
|11.787212|29.005116|      1.123|     28.606|      34.56035|      136.21983|Moderado|  1|  0.0|[11.787212,29.005...|
|11.482636| 29.84576|      1.038|  29.306004|      34.56078|      136.22015|Moderado|  2|  0.0|[11.482636,29.845...|
|11.312064|29.751745|      1.123|  29.006002|      34.55864|      136.22047|Moderado|  3|  0.0|[11.312064,29.751...|
|11.530982|29.505793|      1.404|  29.081003|     34.560654|      136.22006|Moderado|  4|  0.0|[11.530982,29.505...|
|  11.4008|29.422289|      1.504|  29.181004|     34.559814|      136.22006|Moderado|  5|  0.0|[11.4008,29.42228...|
+---------+---------+-----------+-----------+--------------+----

In [ ]:
#calculando a acuracia de classificação na base de testes para o indicador de arvore de decisçaio

#comentar no futuro
from pyspark.ml.classification import DecisionTreeClassifier
import pyspark.sql.functions as func

dt = DecisionTreeClassifier(labelCol='label', featuresCol='features')
modeloDT = dt.fit(dfTreino)

In [ ]:
predicaoDT = modeloDT.transform(dfTeste)
predicaoDT.select(func.col('prediction'), func.col('label'), func.col('id')).show(5)

+----------+-----+---+
|prediction|label| id|
+----------+-----+---+
|       0.0|  0.0|  1|
|       0.0|  0.0|  2|
|       0.0|  0.0|  3|
|       0.0|  0.0|  4|
|       0.0|  0.0|  5|
+----------+-----+---+
only showing top 5 rows



In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction')\
    .evaluate(predicaoDT)

0.9359484629841501